In [2]:
import cirq
import numpy as np
import sympy
from tqdm import tqdm

In [6]:
isinstance([], list)i

True

In [5]:
isinstance(np.array([]), np.ndarray)

True

In [3]:
type(np.array([]))

numpy.ndarray

In [29]:
n_qubits=3
qubits = cirq.GridQubit.rect(1, n_qubits)

def give_gate(ind):
    if 0 <= ind < 2*n_qubits:
        return gates[0]
    elif 2*n_qubits <= ind < 3*n_qubits:
        return gates[1]
    elif 3*n_qubits <= ind < 4*n_qubits:
        return gates[2]
    else:
        return gates[3]
    
def wempty(n_qubits=3):
    return [-1]*5*n_qubits

In [18]:
[[-1]*5*3]*15

[[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]]

In [15]:
def wempty(n_qubits=3):
    return [-1]*5*n_qubits

In [12]:
ac = [-1]*10**5
ac[-1] = 1

def check_list(ac):
    for ind,k in enumerate(ac):
        if k == 1:
            return ind, k
def check_np(ac):
    ind1 = np.where(np.array(ac)==1)[0][0]
    return ind1, ac[ind1]


In [13]:
%timeit check_list(ac)

5 ms ± 137 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
%timeit check_np(ac)

6.69 ms ± 49.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [3]:
indexed_cnots = {}
count=0
for control in range(n_qubits):
    for target in range(n_qubits):
        if control != target:
            indexed_cnots[str(count)] = [control, target]
            count+=1

In [4]:
def append_to_circuit(w,circuit, params):
    ''' Notice we could choose to add multiple gates in parallel (n_qubit gates). 
        We also omit adding identity gate.
        Will explore other options after having a working example.'''
    for ind,inst in enumerate(w):
        if inst == 1:
            if ind < 2*n_qubits:
                control, target = indexed_cnots[str(ind)]
                circuit.append(give_gate(ind).on(qubits[control], qubits[target]))
                return circuit, params
            elif 2*n_qubits <= ind < 3*n_qubits:
                new_param = "th_"+str(len(params))
                params.append(new_param)
                circuit.append(give_gate(ind)(sympy.Symbol(new_param)).on(qubits[int(ind%n_qubits)]))
                return circuit, params
            elif 3*n_qubits <= ind < 5*n_qubits:
                circuit.append(give_gate(ind).on(qubits[int(ind%n_qubits)]))
                return circuit, params
            else:
                return circuit, params

In [5]:
ws =[]
for w in range(14):
    w = wempty()
    action = np.random.choice(range(15))
    w[action] = 1
    ws.append(w)



circuit = []
params = []
for g in ws:
    circuit, params = append_to_circuit(g,circuit, params)
cirq.Circuit(circuit)

(0, 0): ───Rx(-0.5π)───@───────X───────────X──────────Rz(th_0)───Rx(-0.5π)───────────
                       │       │           │
(0, 1): ───────────────X───I───@───@───X───@──────────@──────────Rz(th_2)────X───@───
                                   │   │              │                      │   │
(0, 2): ───────────────────────────X───@───Rz(th_1)───X──────────────────────@───X───

In [566]:
params_values = np.random.random(len(params))
resolver = {k: v for k, v in zip(params, params_values)}
cirq.Circuit(cirq.resolve_parameters(circuit, resolver)) #cirq.unitary( if you want to check the unitay out)

(0, 0): ───I────────────────X───X───Rz(0.13π)───────@───────────Rz(0.14π)───@───
                            │   │                   │                       │
(0, 1): ───Rz(0.133π)───@───┼───┼───@───────────@───X───────────────────────X───
                        │   │   │   │           │
(0, 2): ───Rz(0.077π)───X───@───@───X───────────X───Rx(-0.5π)───Rx(-0.5π)───────

In [567]:
for k in range(len(ws)):
    print(give_gate(np.where(np.array(ws[k]) == 1)[0][0]))

<function rz at 0x7fd8e9456a60>
<function rz at 0x7fd8e9456a60>
CNOT
I
CNOT
CNOT
CNOT
CNOT
Rx(-0.5π)
<function rz at 0x7fd8e9456a60>
CNOT
<function rz at 0x7fd8e9456a60>
CNOT
Rx(-0.5π)


In [26]:
import tensorflow_quantum as tfq

In [31]:

c = cirq.Circuit()
c.append(cirq.CNOT(qubits[0], qubits[1]))

In [45]:
list(c.all_qubits())

[cirq.GridQubit(0, 1), cirq.GridQubit(0, 0)]

In [ ]:
tfq.layers.Expectation()

In [48]:
np.trunc(2/3)

0.0

In [40]:
for k in range(0):
    print(k)

In [27]:
tfq.convert_to_tensor([cirq.X(cirq.GridQubit(0, 0)), cirq.X(cirq.GridQubit(0, 1)), cirq.X(cirq.GridQubit(0, 2))])

<tf.Tensor: shape=(3,), dtype=string, numpy=
array([b'\n\x0f\r\x00\x00\x80?\x1a\x08\n\x030_0\x12\x01X',
       b'\n\x0f\r\x00\x00\x80?\x1a\x08\n\x030_1\x12\x01X',
       b'\n\x0f\r\x00\x00\x80?\x1a\x08\n\x030_2\x12\x01X'], dtype=object)>

In [46]:
import numpy as np

In [47]:
np.math.factorial(10)

3628800

In [654]:
def check_two(w1,w2):
    """
    input w_1, w_2 vectors of length action-space cardinality
    output w_1, w_2 in a "corrected form"
    """
    ind1 = np.where(np.array(w1) == 1)[0][0]
    ind2 = np.where(np.array(w2) == 1)[0][0]
    
    ## (i) both CNOTS, a) same CNOT, b) same targets, keep control on less index qubit first.
    if (ind1 < 2*n_qubits) and (ind2 < 2*n_qubits): 
        if ind1 == ind2:
            w1[ind1] = -1
            w2[ind2] = -1
            control, target = indexed_cnots[str(ind1)]
            w1[4*n_qubits + control] = 1 #this is arbitrary
            w2[4*n_qubits + target] = 1 #this is arbitrary
            return w1, w2
        else:
            control1, target1 = indexed_cnots[str(ind1)]
            control2, target2 = indexed_cnots[str(ind2)]
            if target1 == target2:
                if control1 > control2:
                    return w2, w1
                else:
                    return w1,w2
    
    ## (ii) #both rz, replace second rotation by I
    if (2*n_qubits <= ind1 < 3*n_qubits) and (2*n_qubits <= ind2 < 3*n_qubits): 
        if ind1 == ind2:
            w2[ind1] = -1
            w2[4*n_qubits + ind2%n_qubits] = 1 
            return w1, w2
        elif ind1 > ind2: #put all gates on onequbit first
            return w2, w1
    
    ## (iii) P after CNOT (convention: put P before)
    if (ind1 < 2*n_qubits) and (3*n_qubits <= ind2 < 4*n_qubits): 
        control, target = indexed_cnots[str(ind1)]
        if target == ind2%n_qubits:
            return w2, w1
        
    ## (iv) rz after CNOT (convention: put rz before)
    if (ind1 < 2*n_qubits) and (2*n_qubits <= ind2 < 3*n_qubits): 
        control, target = indexed_cnots[str(ind1)]
        if control == ind2%n_qubits:
            return w2, w1

    ##(v) move CNOTS as much to the right as possible
    if (ind1 < 2*n_qubits) and (2*n_qubits <= ind2 < 5*n_qubits): #
        if ind2%n_qubits not in indexed_cnots[str(ind1)]:
            return w2, w1
        else:
            return w1, w2
        
    ## (vi) try to impose an order on single qubit gates
    if (2*n_qubits <= ind1 < 5*n_qubits) and (2*n_qubits <= ind2 < 5*n_qubits): #
        if ind1%n_qubits > ind2%n_qubits:
            return w2, w1
        else:
            return w1, w2
    ## (vii) like vi but with CNOTS
    #if (ind1 < 2*n_qubits) and (ind2 < 2*n_qubits): #
    #    control1, target1 = indexed_cnots[str(ind1)]
    #    control2, target2 = indexed_cnots[str(ind2)]
        
    #    if (target1 != target2) and (control1 != control2) and (control1 != target2) and (control2 != target1):
    #        if control1 > control2:
    #            return w2, w1
    #        else:
    #            return w1,w2
    ### one could potentially write a vi that takes all identities, but it's better
    #to consider u3
    else:
        return w1,w2

<b> Let's do a check of each case </b>

<b>** (i) Two CNOTS equal **</b>

In [569]:
w1 = wempty()
w1[0] = 1
w2 = wempty()
w2[0] = 1
ws = [w1,w2]

In [570]:
circuit = []
params = []
for g in ws:
    circuit, params = append_to_circuit(g,circuit, params)
cirq.Circuit(circuit)

(0, 0): ───@───@───
           │   │
(0, 1): ───X───X───

In [571]:
w1, w2 = check_two(w1,w2)
ws = [w1,w2]
circuit = []
params = []
for g in ws:
    circuit, params = append_to_circuit(g,circuit, params)
cirq.Circuit(circuit)

(0, 0): ───I───

(0, 1): ───I───

<b>** CNOTS with equal target, control index higher on first **</b> (notice X = target, @ is control)

In [572]:
indexed_cnots

{'0': [0, 1], '1': [0, 2], '2': [1, 0], '3': [1, 2], '4': [2, 0], '5': [2, 1]}

In [573]:
w1 = wempty()
w1[4] = 1
w2 = wempty()
w2[2] = 1
ws = [w1,w2]

circuit = []
params = []
for g in ws:
    circuit, params = append_to_circuit(g,circuit, params)
cirq.Circuit(circuit)

(0, 0): ───X───X───
           │   │
(0, 1): ───┼───@───
           │
(0, 2): ───@───────

In [574]:
w1, w2 = check_two(w1,w2)
circuit = []
params = []
ws = [w1,w2]
for g in ws:
    circuit, params = append_to_circuit(g,circuit, params)
cirq.Circuit(circuit)

(0, 0): ───X───X───
           │   │
(0, 1): ───@───┼───
               │
(0, 2): ───────@───

<b>** both rz, replace second rotation by I **</b>

In [577]:
w1 = wempty()
w1[6] = 1
w2 = wempty()
w2[6] = 1
ws = [w1,w2]

circuit = []
params = []
for g in ws:
    circuit, params = append_to_circuit(g,circuit, params)
cirq.Circuit(circuit)

(0, 0): ───Rz(th_0)───Rz(th_1)───

In [578]:
w1, w2 = check_two(w1,w2)
circuit = []
params = []
ws = [w1,w2]
for g in ws:
    circuit, params = append_to_circuit(g,circuit, params)
cirq.Circuit(circuit)

(0, 0): ───Rz(th_0)───I───

<b> P after CNOT (convention: put P before) </b>

In [581]:
indexed_cnots

{'0': [0, 1], '1': [0, 2], '2': [1, 0], '3': [1, 2], '4': [2, 0], '5': [2, 1]}

In [584]:
cirq.Circuit(cirq.CNOT(qubits[0], qubits[1]))

(0, 0): ───@───
           │
(0, 1): ───X───

In [585]:
w1 = wempty()
w1[0] = 1
w2 = wempty()
w2[10] = 1
ws = [w1,w2]

circuit = []
params = []
for g in ws:
    circuit, params = append_to_circuit(g,circuit, params)
cirq.Circuit(circuit)

(0, 0): ───@───────────────
           │
(0, 1): ───X───Rx(-0.5π)───

In [586]:
w1, w2 = check_two(w1,w2)
circuit = []
params = []
ws = [w1,w2]
for g in ws:
    circuit, params = append_to_circuit(g,circuit, params)
cirq.Circuit(circuit)

(0, 0): ───────────────@───
                       │
(0, 1): ───Rx(-0.5π)───X───

<b> iv) Moving CNOTS to the right in non interacting qubits</b> (we do this so we can still use the comparison between 2)

In [656]:
w1 = wempty()
w1[0] = 1
w2 = wempty()
w2[8] = 1
w3 = wempty()
w3[11]=1
ws = [w1,w2,w3]

circuit = []
params = []
for g in ws:
    circuit, params = append_to_circuit(g,circuit, params)
cirq.Circuit(circuit)

(0, 0): ───@──────────────────────
           │
(0, 1): ───X──────────────────────

(0, 2): ───Rz(th_0)───Rx(-0.5π)───

In [657]:
for k in range(len(ws)):
    print(give_gate(np.where(np.array(ws[k]) == 1)[0][0]))

CNOT
<function rz at 0x7fd8e9456a60>
Rx(-0.5π)


In [659]:
ws_previous = ws

for k in range(2):

    circuit = []
    params = []
    for g in ws_previous:
        circuit, params = append_to_circuit(g,circuit, params)
    print(cirq.Circuit(circuit))
    print("************************************************************************************************************\n\n")


    ws_odd=[]
    for d in range(0,len(ws_previous),2):
        if d<len(ws_previous)-1:
            w1 = ws_previous[d]
            w2 = ws_previous[d+1]
            wc1, wc2 = check_two(w1, w2)
            ws_odd.append(wc1)
            ws_odd.append(wc2)
        else:
            w1 = ws_previous[d]
            ws_odd.append(w1)
    circuit = []
    params = []
    for g in ws_odd:
        circuit, params = append_to_circuit(g,circuit, params)
    print(cirq.Circuit(circuit))
    print("************************************************************************************************************\n\n")


    ws_final = [ws_odd[0]]
    for d in range(1, len(ws_odd), 2):
        if d<len(ws_odd)-1:
            w1 = ws_odd[d]
            w2 = ws_odd[d+1]
            wc1, wc2 = check_two(w1, w2)
            ws_final.append(wc1)
            ws_final.append(wc2)
        else:
            w1 = ws_odd[d]
            ws_final.append(w1)
    ws_previous = ws_final
    ws_final = []

(0, 0): ───@──────────────────────
           │
(0, 1): ───X──────────────────────

(0, 2): ───Rz(th_0)───Rx(-0.5π)───
************************************************************************************************************


(0, 0): ───@──────────────────────
           │
(0, 1): ───X──────────────────────

(0, 2): ───Rz(th_0)───Rx(-0.5π)───
************************************************************************************************************


(0, 0): ───@──────────────────────
           │
(0, 1): ───X──────────────────────

(0, 2): ───Rz(th_0)───Rx(-0.5π)───
************************************************************************************************************


(0, 0): ───@──────────────────────
           │
(0, 1): ───X──────────────────────

(0, 2): ───Rz(th_0)───Rx(-0.5π)───
************************************************************************************************************




In [643]:
for k in range(len(ws_previous)):
    print(give_gate(np.where(np.array(ws_previous[k]) == 1)[0][0]))

<function rz at 0x7fd8e9456a60>
Rx(-0.5π)
CNOT


<b> Let's check how this works on a random circuit </b>

In [681]:
def check_and_recheck(instructions, its=100, printing=False):
    """
    instructions:: collection of vectors of length |action_space|, eachone with information of only
    one qubit gate (or a cnot). 
    
    its:: # of iterations
    prining:: boolean, visualize evolution
    
    we take blocks of two consecutive gates, beggining interchaning when to begin.
        """
    ws_previous = instructions

    for k in range(10):
        if printing:
            circuit = []
            params = []
            for g in ws_previous:
                circuit, params = append_to_circuit(g,circuit, params)
            print(cirq.Circuit(circuit))
            print("************************************************************************************************************\n\n")

        ws_odd=[]
        for d in range(0,len(ws_previous),2):
            if d<len(ws_previous)-1:
                w1 = ws_previous[d]
                w2 = ws_previous[d+1]
                wc1, wc2 = check_two(w1, w2)
                ws_odd.append(wc1)
                ws_odd.append(wc2)
            else:
                w1 = ws_previous[d]
                ws_odd.append(w1)
        if printing:

            circuit = []
            params = []
            for g in ws_odd:
                circuit, params = append_to_circuit(g,circuit, params)
            print(cirq.Circuit(circuit))
            print("************************************************************************************************************\n\n")


        ws_final = [ws_odd[0]]
        for d in range(1, len(ws_odd), 2):
            if d<len(ws_odd)-1:
                w1 = ws_odd[d]
                w2 = ws_odd[d+1]
                wc1, wc2 = check_two(w1, w2)
                ws_final.append(wc1)
                ws_final.append(wc2)
            else:
                w1 = ws_odd[d]
                ws_final.append(w1)
        ws_previous = ws_final
        ws_final = []
    return ws_previous
    

In [19]:
for k in range(100):
    if k == 10:
        print(k)
        break
    else:
        print(k)

0
1
2
3
4
5
6
7
8
9
10


[8, 11, 7, 5, 7, 2, 6, 4, 11, 2, 11, 14, 0, 7, 4, 10, 3, 0, 11, 0]

In [684]:
ws =[]
for w in range(5):
    w = wempty()
    action = np.random.choice(range(15))
    w[action] = 1
    ws.append(w)

circuit = []
params = []
for g in ws:
    circuit, params = append_to_circuit(g,circuit, params)
cirq.Circuit(circuit)

(0, 0): ───I──────────@──────────────────
                      │
(0, 1): ───Rz(th_1)───┼──────────────────
                      │
(0, 2): ──────────────X───Rz(th_0)───I───

In [686]:
new_circuit = check_and_recheck(ws,printing=True)

newcircuit = []
params = []
for g in new_circuit:
    circuit, params = append_to_circuit(g,newcircuit, params)
if len(params)==0:
    cirq.unitary(cirq.Circuit(newcircuit)) - cirq.unitary(cirq.Circuit(circuit)) 


(0, 0): ───I──────────@──────────────────
                      │
(0, 1): ───Rz(th_1)───┼──────────────────
                      │
(0, 2): ──────────────X───Rz(th_0)───I───
************************************************************************************************************


(0, 0): ───I──────────@──────────────────
                      │
(0, 1): ───Rz(th_0)───┼──────────────────
                      │
(0, 2): ──────────────X───Rz(th_1)───I───
************************************************************************************************************


(0, 0): ───I──────────@──────────────────
                      │
(0, 1): ───Rz(th_0)───┼──────────────────
                      │
(0, 2): ──────────────X───Rz(th_1)───I───
************************************************************************************************************


(0, 0): ───I──────────@──────────────────
                      │
(0, 1): ───Rz(th_0)───┼──────────────────
                      │
(0, 2): ─────

Another example..

In [669]:
ws =[]
for w in range(10):
    w = wempty()
    action = np.random.choice(range(15))
    w[action] = 1
    ws.append(w)

circuit = []
params = []
for g in ws:
    circuit, params = append_to_circuit(g,circuit, params)
cirq.Circuit(circuit)

(0, 0): ──────────────────────────────────@──────────X───────@───
                                          │          │       │
(0, 1): ───Rx(-0.5π)───I───Rz(th_0)───X───X──────────@───────┼───
                                      │                      │
(0, 2): ──────────────────────────────@───Rz(th_1)───I───I───X───

In [671]:
new_circuit = check_and_recheck(ws)
newcircuit = []
params = []
for g in new_circuit:
    circuit, params = append_to_circuit(g,newcircuit, params)

(0, 0): ──────────────────────────────────@──────────X───────@───
                                          │          │       │
(0, 1): ───Rx(-0.5π)───I───Rz(th_0)───X───X──────────@───────┼───
                                      │                      │
(0, 2): ──────────────────────────────@───Rz(th_1)───I───I───X───
************************************************************************************************************


(0, 0): ──────────────────────────────────@──────────X───────@───
                                          │          │       │
(0, 1): ───Rx(-0.5π)───I───Rz(th_0)───X───X──────────@───────┼───
                                      │                      │
(0, 2): ──────────────────────────────@───Rz(th_1)───I───I───X───
************************************************************************************************************


(0, 0): ──────────────────────────────────@───X───@───
                                          │   │   │
(0, 1): ───Rx(-0.5π)───

Final example (and check the ordering in final list)

In [687]:
ws =[]
for w in range(15):
    w = wempty()
    action = np.random.choice(range(15))
    w[action] = 1
    ws.append(w)

circuit = []
params = []
for g in ws:
    circuit, params = append_to_circuit(g,circuit, params)
    print(circuit[-1])
cirq.Circuit(circuit)

Rz(th_0/piπ)((0, 1))
Rz(th_1/piπ)((0, 2))
CNOT((0, 2), (0, 0))
CNOT((0, 2), (0, 1))
CNOT((0, 2), (0, 0))
CNOT((0, 1), (0, 2))
Rz(th_2/piπ)((0, 1))
Rx(-0.5π)((0, 0))
CNOT((0, 2), (0, 0))
I((0, 2))
CNOT((0, 2), (0, 0))
CNOT((0, 2), (0, 1))
Rz(th_3/piπ)((0, 1))
CNOT((0, 1), (0, 2))
CNOT((0, 0), (0, 2))


┌─────────┐
(0, 0): ──────────────X───────X───Rx(-0.5π)────────────X────────X──────────────────────@───
                      │       │                        │        │                      │
(0, 1): ───Rz(th_0)───┼───X───┼───@────────────Rz(th_2)┼────────┼───X───Rz(th_3)───@───┼───
                      │   │   │   │                    │        │   │              │   │
(0, 2): ───Rz(th_1)───@───@───@───X────────────────────@────I───@───@──────────────X───X───
                                              └─────────┘

In [689]:
new_circuit = check_and_recheck(ws)
newcircuit = []
params = []
for g in new_circuit:
    circuit, params = append_to_circuit(g,newcircuit, params)
    print(circuit[-1])
cirq.Circuit(circuit)

Rx(-0.5π)((0, 0))
Rz(th_0/piπ)((0, 1))
Rz(th_1/piπ)((0, 2))
CNOT((0, 2), (0, 0))
CNOT((0, 2), (0, 1))
Rz(th_2/piπ)((0, 1))
CNOT((0, 2), (0, 0))
CNOT((0, 1), (0, 2))
CNOT((0, 2), (0, 0))
I((0, 2))
CNOT((0, 2), (0, 0))
CNOT((0, 2), (0, 1))
Rz(th_3/piπ)((0, 1))
CNOT((0, 0), (0, 2))
CNOT((0, 1), (0, 2))


┌─────────┐                       ┌─────────┐
(0, 0): ───Rx(-0.5π)───X────────────────X────────X───────X────────────────@────────
                       │                │        │       │                │
(0, 1): ───Rz(th_0)────┼───X────Rz(th_2)┼────@───┼───────┼───X────Rz(th_3)┼────@───
                       │   │            │    │   │       │   │            │    │
(0, 2): ───Rz(th_1)────@───@────────────@────X───@───I───@───@────────────X────X───
                               └─────────┘                       └─────────┘

In [773]:
ws =[]
for w in range(10**3):
    w = wempty()
    action = np.random.choice(range(15))
    w[action] = 1
    ws.append(w)

circuit = []
params = []
for g in ws:
    circuit, params = append_to_circuit(g,circuit, params)
    print(circuit[-1])
cirq.Circuit(circuit)
print("********************************\n\n")

new_circuit = check_and_recheck(ws, its=10**6)
newcircuit = []
params = []
for g in new_circuit:
    circuit, params = append_to_circuit(g,newcircuit, params)
    print(circuit[-1])
cirq.Circuit(circuit)

CNOT((0, 1), (0, 2))
I((0, 2))
Rx(-0.5π)((0, 1))
Rx(-0.5π)((0, 2))
CNOT((0, 0), (0, 1))
I((0, 0))
CNOT((0, 2), (0, 0))
CNOT((0, 0), (0, 1))
I((0, 2))
Rz(th_0/piπ)((0, 2))
CNOT((0, 2), (0, 0))
Rx(-0.5π)((0, 0))
I((0, 0))
I((0, 2))
Rz(th_1/piπ)((0, 0))
Rz(th_2/piπ)((0, 0))
CNOT((0, 2), (0, 0))
I((0, 2))
Rz(th_3/piπ)((0, 2))
I((0, 2))
CNOT((0, 1), (0, 0))
CNOT((0, 0), (0, 1))
Rz(th_4/piπ)((0, 1))
Rz(th_5/piπ)((0, 2))
Rz(th_6/piπ)((0, 1))
Rz(th_7/piπ)((0, 0))
CNOT((0, 1), (0, 2))
Rx(-0.5π)((0, 0))
CNOT((0, 0), (0, 1))
Rz(th_8/piπ)((0, 0))
I((0, 0))
I((0, 1))
CNOT((0, 0), (0, 1))
CNOT((0, 1), (0, 2))
Rz(th_9/piπ)((0, 0))
CNOT((0, 1), (0, 2))
CNOT((0, 2), (0, 0))
I((0, 1))
CNOT((0, 2), (0, 1))
I((0, 2))
CNOT((0, 2), (0, 1))
CNOT((0, 0), (0, 1))
CNOT((0, 1), (0, 0))
CNOT((0, 2), (0, 1))
CNOT((0, 0), (0, 1))
CNOT((0, 1), (0, 0))
Rx(-0.5π)((0, 2))
Rx(-0.5π)((0, 2))
Rx(-0.5π)((0, 1))
I((0, 0))
I((0, 0))
CNOT((0, 1), (0, 0))
Rx(-0.5π)((0, 2))
Rx(-0.5π)((0, 1))
CNOT((0, 1), (0, 2))
CNOT((0, 0), (0

CNOT((0, 1), (0, 2))
Rx(-0.5π)((0, 1))
I((0, 2))
Rx(-0.5π)((0, 2))
CNOT((0, 0), (0, 1))
I((0, 0))
CNOT((0, 2), (0, 0))
I((0, 2))
CNOT((0, 0), (0, 1))
Rx(-0.5π)((0, 0))
Rz(th_0/piπ)((0, 2))
CNOT((0, 2), (0, 0))
I((0, 0))
Rz(th_1/piπ)((0, 0))
I((0, 0))
I((0, 2))
CNOT((0, 2), (0, 0))
I((0, 2))
Rz(th_2/piπ)((0, 2))
I((0, 2))
CNOT((0, 1), (0, 0))
Rz(th_3/piπ)((0, 0))
CNOT((0, 0), (0, 1))
Rx(-0.5π)((0, 0))
Rz(th_4/piπ)((0, 0))
Rz(th_5/piπ)((0, 1))
I((0, 1))
Rz(th_6/piπ)((0, 2))
CNOT((0, 1), (0, 2))
CNOT((0, 0), (0, 1))
I((0, 0))
Rz(th_7/piπ)((0, 0))
I((0, 1))
CNOT((0, 0), (0, 1))
I((0, 1))
I((0, 1))
I((0, 2))
CNOT((0, 2), (0, 0))
CNOT((0, 2), (0, 1))
I((0, 2))
CNOT((0, 0), (0, 1))
CNOT((0, 2), (0, 1))
CNOT((0, 1), (0, 0))
CNOT((0, 0), (0, 1))
CNOT((0, 2), (0, 1))
Rx(-0.5π)((0, 2))
CNOT((0, 1), (0, 0))
I((0, 0))
I((0, 0))
Rx(-0.5π)((0, 1))
Rx(-0.5π)((0, 2))
CNOT((0, 1), (0, 0))
Rz(th_8/piπ)((0, 0))
Rx(-0.5π)((0, 1))
Rx(-0.5π)((0, 2))
CNOT((0, 1), (0, 2))
CNOT((0, 0), (0, 1))
CNOT((0, 0), (0, 

┌──┐                                                                                                                                                                                                              ┌──────────┐                                                                                                                                                   ┌──┐                                                                                                                                                                                                                       ┌──────────┐                                                                           ┌──┐                       ┌──┐                                                                   ┌──────────┐                                       ┌──────────┐                                                                                                                                               ┌──┐                                                                                                                                                                   ┌──────────┐                                                                                                                           ┌──────────┐                                                   ┌──┐   ┌──────────┐                                                   ┌──────────┐                                                                               ┌──────────┐                                                                           ┌──────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                       ┌──────────┐                           ┌──┐                                                                                                                                                                                                                           ┌──────────┐                                                                                                                                                                                                                               ┌──┐                                                                                                                           ┌──┐                                                                               ┌──┐                                                                                                                                                                                                                     ┌───────────┐                        ┌───────────┐                                                            ┌──┐                                                                                                                                                                                                                                                                            ┌──┐           ┌───────────┐                                                                         ┌──┐                                                                                                                                                                                    ┌──────────┐                                                                                      ┌──┐   ┌──┐               ┌───────────┐                                                                                                                                                           ┌───────────┐                                    ┌──┐     

Notice that all 1 qubit gates are as ordered as possible... 
This clears the room to implement the following:

we'll check if a sequence u3 = Rz P Rz P Rz is encountered 
(hence this will suffice to describe more general rotation on 1qubit and
any consecutive Rz/P can be absorbed in u3).

Question: is there any smart way to use u2?

In [774]:
#check the number of consecutives... (nice way to visualize the circuit! :D )
iss, jss = np.where((np.array(new_circuit)+1)/2 == 1.)
c=0
for i,j in zip(iss, jss):
    if j>2*n_qubits:
        print(j%n_qubits, c)
        c+=1
    else:
        c=0
        print("\n")



1 0
2 1
2 2


0 0


2 0


0 0
2 1


0 0


0 0
2 1


2 0
2 1
2 2






0 0


1 0
1 1
2 2




0 0


1 0


1 0
1 1
2 2




2 0










2 0


0 0
0 1
1 2
2 3




1 0
2 1














2 0




0 0
2 1


2 0


0 0
2 1
2 2








0 0
0 1


1 0
2 1


0 0
1 1


2 0
2 1


1 0
2 1
2 2


0 0
0 1
1 2


2 0








2 0


0 0




1 0
2 1
2 2




1 0


1 0
1 1
1 2






1 0
1 1
1 2
1 3
2 4
2 5


2 0
2 1
2 2


0 0
1 1
1 2


1 0
2 1
2 2
2 3
2 4
2 5




0 0


0 0
0 1
0 2
0 3
0 4
1 5
2 6


1 0
2 1
2 2






0 0
0 1
0 2
0 3
1 4
1 5


2 0
2 1
2 2
2 3
2 4


1 0








1 0
2 1




0 0
0 1
0 2
1 3
1 4


1 0
1 1
2 2
2 3
2 4
2 5




0 0
0 1
1 2
1 3
2 4


1 0
1 1
2 2






1 0
2 1




0 0


1 0
1 1




2 0


0 0
2 1


0 0


0 0


1 0
1 1
1 2
1 3
2 4


1 0


0 0
0 1
1 2


2 0
2 1












0 0
2 1
2 2


0 0


1 0
2 1




0 0
0 1
0 2
2 3
2 4


1 0


0 0


0 0
0 1
1 2
2 3


1 0
1 1
2 2


0 0
0 1
0 2
0 3
1 4
1 5






1 0




1 0
1 1




0 0
1 1




0 0
0 1
0 2
1 3
1 4
1 5
1 6
2 7
2 8










0 0
1 1
1 2
2 

In [775]:
def rolling(a, window):
    shape = (a.size - window + 1, window)
    strides = (a.itemsize, a.itemsize)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)


#check the number of consecutives... (nice way to visualize the circuit! :D )
cropped_circuit = np.array(new_circuit)
iss, jss = np.where((np.array(new_circuit)+1)/2 == 1.)
c=0
internal_count_wire=0
u3_seq = np.array([2,3,2,3,2])

for i,j in zip(iss, jss):
    if j>2*n_qubits:
        #print(j%n_qubits, give_gate(j), j/n_qubits)
        while internal_count_wire == 0:
            internal_count_wire +=1
            qindfav = j%n_qubits
            string_to_eval=[]
            indexes_saving = []
            
        if j%n_qubits == qindfav:
            string_to_eval.append(int(np.trunc(j/n_qubits)))
            indexes_saving.append(i)
            internal_count_wire+=1
            
        else:
            if internal_count_wire > 5:
                print(u3_seq)
                print(rolling(np.array(string_to_eval), 5))
                print("\n \n")
                if u3_seq in rolling(np.array(string_to_eval), 5): #this is u3
                    ind=0
                    for gind in indexes_saving: #erase everyone
                        cropped_circuit[gind] = -1 #erase everyone
                        if ind <5:
                            cropped_circuit[gind][u3_seq[ind]*n_qubits + qindfav] = 1 #add each element of u3_seq
                            ind+=1
                        else:
                            cropped_circuit[gind][4*n_qubits + qindfav] = 1 #add identity
                            
            internal_count_wire=0
            string_to_eval=[]
            indexes_saving = []
        c+=1
    else:
        internal_count_wire=0
        c=0
np.mean(cropped_circuit - new_circuit)

[2 3 2 3 2]
[[4 4 4 3 3]]

 

[2 3 2 3 2]
[[4 3 2 4 4]
 [3 2 4 4 4]
 [2 4 4 4 2]]

 

[2 3 2 3 2]
[[2 3 4 3 2]]

 

[2 3 2 3 2]
[[4 3 4 3 4]]

 

[2 3 2 3 2]
[[3 3 4 2 4]
 [3 4 2 4 2]]

 



0.0

In [21]:
np.array(wempty())

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

In [855]:
def detect_u3_and_reduce(ws):
    cropped_circuit = np.array(ws)
    iss, jss = np.where((np.array(ws)+1)/2 == 1.) #all indices with information of gate
    c=0
    internal_count_wire=0
    u3_seq = np.array([2,3,2,3,2])

    for i,j in zip(iss, jss):
        if j>=2*n_qubits: #I don't want CNOTs
            while internal_count_wire == 0:
                internal_count_wire +=1
                qindfav = j%n_qubits
                string_to_eval=[]
                indexes_saving = []

            if (j%n_qubits == qindfav):
                string_to_eval.append(int(np.trunc(j/n_qubits)))
                indexes_saving.append(i)
                internal_count_wire+=1
                
                if (i == iss[-1]): #it can happen that it's at one extreme
                    print("internal_count_wire")
                    if internal_count_wire > 5:
                        if u3_seq in rolling(np.array(string_to_eval), 5): #this is u3
                            ind=0
                            for gind in indexes_saving: #erase everyone
                                cropped_circuit[gind] = -1 #erase everyone
                                if ind <5:
                                    cropped_circuit[gind][u3_seq[ind]*n_qubits + qindfav] = 1 #add each element of u3_seq
                                    ind+=1
                                else:
                                    cropped_circuit[gind][4*n_qubits + qindfav] = 1 #add identity

                    internal_count_wire=0
                    string_to_eval=[]
                    indexes_saving = []
            else: 
                if internal_count_wire > 5:
                    if u3_seq in rolling(np.array(string_to_eval), 5): #this is u3
                        ind=0
                        for gind in indexes_saving: #erase everyone
                            cropped_circuit[gind] = -1 #erase everyone
                            if ind <5:
                                cropped_circuit[gind][u3_seq[ind]*n_qubits + qindfav] = 1 #add each element of u3_seq
                                ind+=1
                            else:
                                cropped_circuit[gind][4*n_qubits + qindfav] = 1 #add identity
        
                internal_count_wire=0
                string_to_eval=[]
                indexes_saving = []
            c+=1
        else:
            internal_count_wire=0
            c=0
    return cropped_circuit

In [786]:
ws =[]
for w in range(10**4):
    w = wempty()
    action = np.random.choice(range(6,13,1))
    w[action] = 1
    ws.append(w)
new_circuit = check_and_recheck(ws, its=10**6)

In [861]:
news = detect_u3_and_reduce(new_circuit)

0 9 0 <function rz at 0x7fd8e9456a60> 2.0 0
1 9 0 Rx(-0.5π) 3.0 2
2 9 0 <function rz at 0x7fd8e9456a60> 2.0 3
3 9 0 Rx(-0.5π) 3.0 4
4 9 0 <function rz at 0x7fd8e9456a60> 2.0 5
5 9 0 Rx(-0.5π) 3.0 6
6 9 0 <function rz at 0x7fd8e9456a60> 2.0 7
7 9 0 Rx(-0.5π) 3.0 8
8 9 0 <function rz at 0x7fd8e9456a60> 2.0 9
9 9 0 Rx(-0.5π) 3.0 10
internal_count_wire


In [828]:
ws =[]
for k in range(10):
    w=wempty()
    if k%2 == 0:
        w[6] = 1
    else:
        w[9] = 1
    ws.append(w)

circuit = []
params = []
for g in ws:
    circuit, params = append_to_circuit(g,circuit, params)
    print(circuit[-1])
cirq.Circuit(circuit)

Rz(th_0/piπ)((0, 0))
Rx(-0.5π)((0, 0))
Rz(th_1/piπ)((0, 0))
Rx(-0.5π)((0, 0))
Rz(th_2/piπ)((0, 0))
Rx(-0.5π)((0, 0))
Rz(th_3/piπ)((0, 0))
Rx(-0.5π)((0, 0))
Rz(th_4/piπ)((0, 0))
Rx(-0.5π)((0, 0))


(0, 0): ───Rz(th_0)───Rx(-0.5π)───Rz(th_1)───Rx(-0.5π)───Rz(th_2)───Rx(-0.5π)───Rz(th_3)───Rx(-0.5π)───Rz(th_4)───Rx(-0.5π)───

In [829]:
new_circuit = check_and_recheck(ws, its=10**6)

circuit = []
params = []
for g in new_circuit:
    circuit, params = append_to_circuit(g,circuit, params)
    print(circuit[-1])
cirq.Circuit(circuit)

Rz(th_0/piπ)((0, 0))
Rx(-0.5π)((0, 0))
Rz(th_1/piπ)((0, 0))
Rx(-0.5π)((0, 0))
Rz(th_2/piπ)((0, 0))
Rx(-0.5π)((0, 0))
Rz(th_3/piπ)((0, 0))
Rx(-0.5π)((0, 0))
Rz(th_4/piπ)((0, 0))
Rx(-0.5π)((0, 0))


(0, 0): ───Rz(th_0)───Rx(-0.5π)───Rz(th_1)───Rx(-0.5π)───Rz(th_2)───Rx(-0.5π)───Rz(th_3)───Rx(-0.5π)───Rz(th_4)───Rx(-0.5π)───

In [859]:
circuit = []
params = []
for g in news:
    circuit, params = append_to_circuit(g,circuit, params)
    print(circuit[-1])
cirq.Circuit(circuit)

Rz(th_0/piπ)((0, 0))
Rx(-0.5π)((0, 0))
Rz(th_1/piπ)((0, 0))
Rx(-0.5π)((0, 0))
Rz(th_2/piπ)((0, 0))
I((0, 0))
I((0, 0))
I((0, 0))
I((0, 0))
I((0, 0))


(0, 0): ───Rz(th_0)───Rx(-0.5π)───Rz(th_1)───Rx(-0.5π)───Rz(th_2)───I───I───I───I───I───